# Highly divisible triangular number

[problem 12](https://projecteuler.net/problem=12)
> The sequence of triangle numbers is generated by adding the natural numbers. So the 7th triangle number would be 

> $$1 + 2 + 3 + 4 + 5 + 6 + 7 = 28$$

> The first ten terms would be:

> $$1, 3, 6, 10, 15, 21, 28, 36, 45, 55, ...$$

> Let us list the factors of the first seven triangle numbers:

> \begin{align}
1&:& 1 \cr
3&:& 1,3 \cr
6&:& 1,2,3,6 \cr
10&:& 1,2,5,10 \cr
15&:& 1,3,5,15 \cr
21&:& 1,3,7,21 \cr
28&:& 1,2,4,7,14,28 \cr
\end{align}

> We can see that 28 is the first triangle number to have over five divisors.

> What is the value of the first triangle number to have over five hundred divisors?

### Triangle numbers

Naive implementation would iterate and sum all natural numbers up to n to get the nth triangle number.

In [1]:
def naiveTriangleN(n):
    return sum(range(n+1))

print(naiveTriangleN(7))
%timeit naiveTriangleN(7)

print(naiveTriangleN(10**6))
%timeit naiveTriangleN(10**6)

28
1000000 loops, best of 3: 748 ns per loop
500000500000
10 loops, best of 3: 28.1 ms per loop


The sum of n numbers with a constant delta is called an arithmetic series. We learned in problem 1 that given: $n$, $a_1$, and $a_n$, we can calculate the value of an artithmetic series:

$$S_n = \frac {n(a_1+a_n)}2$$

That means for triangle numbers:

$$T_n = \frac {n(n+1)}2$$

---

$$1,3,6,10...$$

\begin{align}
T_1 & = \frac {1\times(1+1)}2 = \frac {1\times(2)}2 = 1 \cr
T_2 & = \frac {2\times(2+1)}2 = \frac {2\times(3)}2 = 3 \cr
T_3 & = \frac {3\times(3+1)}2 = \frac {3\times(4)}2 = 6 \cr
T_4 & = \frac {4\times(4+1)}2 = \frac {4\times(5)}2 = 10 \cr
\end{align}

In [2]:
def seriesTriangleN(n):
    return n * (n + 1) // 2
  
print(seriesTriangleN(7))
%timeit seriesTriangleN(7)

print(seriesTriangleN(10**6))
%timeit seriesTriangleN(10**6)

28
The slowest run took 4.06 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 257 ns per loop
500000500000
1000000 loops, best of 3: 303 ns per loop


### Number of Factors

Naive implementation checks all natural numbers below $n$ to see how many are factors. 

In [3]:
def naiveNumFactors(n):
    return sum(n % i == 0 for i in range(1, n+1))
    
print(naiveNumFactors(28))

6


In [4]:
print(naiveNumFactors(16))
%timeit naiveNumFactors(16)

print(naiveNumFactors(10**6))
%timeit naiveNumFactors(10**6)

5
100000 loops, best of 3: 4.98 µs per loop
49
1 loops, best of 3: 213 ms per loop


In Problem 5 we learned that any number $n$ can be decomposed using prime numbers.

$$n = p^\alpha \times q^\beta \times ... r^\gamma$$

It is also true that any number, $m$, that is a power of a prime, $p$, $m=p^\alpha$, $m$ will have $\alpha+1$ factors. For example:

\begin{align}
5 & = 5^1, \text{ factors } \mapsto 1, 5 & \mapsto & 5^0, 5^1 \cr
8 & = 2^3, \text{ factors }  \mapsto 1, 2, 4, 8 & \mapsto & 2^0, 2^1, 2^2, 2^3 \cr
9 & = 3^2, \text{ factors }  \mapsto 1, 3, 9 & \mapsto & 3^0, 3^1, 3^2
\end{align}

Now consider a number that is composed of multiple primes:

$$n = p^\alpha \times q^\beta \times ... r^\gamma$$

The number of total divisors can be determined by the powers of the prime decomposition using the [rule of product](http://en.wikipedia.org/wiki/Rule_of_product), so $n$ will have:

$$\text{num_factors} = (\alpha+1)\times(\beta+1)\times...(\gamma+1)$$

In [5]:
def eratosthenes(n):
    """ Returns a list of primes < n """
    sieve = [True] * (n//2)
    end = len(sieve)
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[i//2]:
            start = i*i//2
            sieve[start::i] = [False] * ((end-start-1)//i + 1)
    return ([2] if n > 2 else []) + [2*i+1 for i in range(1, n//2) if sieve[i]]

class FactorService:
    def __init__(self, primes):
        self.primes = primes
        
    def primeFactors(self, n):
        factors = {}
        for p in self.primes:
            if p*p > n:
                break
            power = 0
            while not n % p:
                power += 1
                n //= p
            if power > 0: factors[p] = power
        if n != 1: factors[n] = 1
        return factors
    
    def numFactors(self, n):
        pf = self.primeFactors(n)
        nf = 1
        for f, c in pf.items():
            nf *= (c+1)
        return nf
    
myFactorService = FactorService(eratosthenes(65536))

print(2*2*3*3*5*7*7, myFactorService.primeFactors(2*2*3*3*5*7*7))
print(28, myFactorService.primeFactors(28))
print(myFactorService.numFactors(28))

8820 {2: 2, 3: 2, 5: 1, 7: 2}
28 {2: 2, 7: 1}
6


In [6]:
print(myFactorService.numFactors(16))
%timeit myFactorService.numFactors(16)

print(myFactorService.numFactors(10**6))
%timeit myFactorService.numFactors(10**6)

5
100000 loops, best of 3: 2.73 µs per loop
49
100000 loops, best of 3: 5.92 µs per loop


## Solution

In [7]:
def euler12BruteForce():
    i = 1
    while True:
        tN = seriesTriangleN(i)
        if myFactorService.numFactors(tN) > 500:
            return tN
        i+= 1
        
print(euler12BruteForce())
%timeit euler12BruteForce()

76576500
1 loops, best of 3: 228 ms per loop


Looking again at the generalization for the triangle numbers

$$T_n = \frac{n (n+1)}2$$

we can use the fact that $n$ and $n+1$ are co-prime, meaning they do not share any common prime factors. Because of this, we can generalize the number of total divisors for a triangle number as:

$$D(t) = D\left(\frac n2\right) \times D\left(n+1\right)$$ 

if $n$ is even.

$$D(t) = D\left(n\right) \times D\left(\frac {n+1}2\right)$$ 

if $n+1$ is even.

In [8]:
def memoizedFactorCounter():
    memo = {}
    def numFactors(n):
        if n not in memo:
            memo[n] = myFactorService.numFactors(n)
        return memo[n]
    return numFactors

def euler12Math():
    numFactors = memoizedFactorCounter()
    n = 0
    factors = -1
    while factors <= 500:
        n += 1
        if n % 2:
            factors = numFactors(n) * numFactors((n+1)//2)
        else:
            factors = numFactors(n//2) * numFactors(n+1)
    return seriesTriangleN(n)
    
print(euler12Math())
%timeit euler12Math()

76576500
10 loops, best of 3: 60.9 ms per loop
